**Table of contents**<a id='toc0_'></a>    
- [model](#toc1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
import numpy as np
import re
from IPython.display import display, HTML
import sys
import os

import warnings
warnings.filterwarnings("ignore")
sys.path.append('/Users/jianbinlin/Documents/Master-Data-Spreadsheet/script/')

import utility
from utility import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

display(HTML("<style>.container { width:50% !important; }</style>"))
#display(HTML('<style>.output { max-width:800px !important; }</style>')) # control output width
display(HTML("<style>.container { width:130% !important; }</style>"))

pd.set_option('display.max_columns', None)   # Show number of columns, None means show all columns
pd.set_option('display.max_rows', 50)        # show top and bottom 15, middle with ......
pd.set_option('display.width', None)         # Set the width of the display
pd.set_option('display.max_colwidth', None)  # Show full content of each column

In [2]:
# house_price = pd.read_csv('https://raw.githubusercontent.com/KevinJianLin/House_Price_Advanced_Regression/refs/heads/main/Data/train.csv')
filepath1 = '/Users/jianbinlin/Documents/Scientific_2022_Jan.csv'
house_ON = pd.read_csv(filepath1).iloc[5:,110:131].reset_index(drop=True)
house_ON.columns = house_ON.iloc[0]
house_ON = house_ON[1:].reset_index(drop=True)
house_ON = house_ON[(house_ON['comments'].str.contains("relevant")) &  ~(house_ON['comments'].isna())].reset_index(drop=True)

# house_ON["sold price\n(million)"] = house_ON[['asking price\n(million)', 'sold price\n(million)']].apply(
#     lambda x: x['sold price\n(million)'] if not pd.isna(x['sold price\n(million)']) else x['asking price\n(million)'], axis=1)

house_ON.drop(columns=['Listing Days','asking price\n(million)'], inplace=True)  # drop comments column
house_ON.dropna(subset=['sold price\n(million)'], inplace=True)  # drop rows with no sold price
# house_ON['built age'] = house_ON['built age'].astype(int) 
house_ON['sold year'] = pd.to_datetime(house_ON['sold year'])


data_set = house_ON.copy()
data_set.dtypes
data_set.head(5)


0
built age                        object
sold year                datetime64[ns]
Municipality                     object
size ft^2 | ,m^2                 object
sold price\n(million)            object
lot size \n(feet 2)              object
structure                        object
cross street                     object
address1                         object
address2                         object
basement Type                    object
postal code                      object
Kitchen                          object
bathroom                         object
Bedrooms                         object
Rooms                            object
construction                     object
community                        object
comments                         object
dtype: object

,built age,sold year,Municipality,"size ft^2 | ,m^2",sold price\n(million),lot size \n(feet 2),structure,cross street,address1,address2,basement Type,postal code,Kitchen,bathroom,Bedrooms,Rooms,construction,community,comments
0,NaN,2025-04-03,Markham,2800,1.82,NaN,NaN,NaN,38,Lancashire,NaN,L3R 8K1,NaN,NaN,NaN,NaN,NaN,Unionville,relevant
5,32,2025-06-03,Markham,3247,1.98,NaN,NaN,NaN,29,Webber Cres,NaN,L3R 2L3,NaN,NaN,NaN,NaN,NaN,Unionville,"relevant, sep entrance from backyead; basement is above ground"
9,42,2025-05-03,Markham,2950,1.8,4229.2366,NaN,NaN,21,Bauer Cres,finished/full,L3R 4H3,1,4,NaN,NaN,NaN,Unionville,relevant
10,52,2025-04-03,Markham,2581,1.9,6488.475,NaN,NaN,23,Hagerman Blvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unionville,relevant
11,46,2025-04-03,Markham,3213,2.07,7500,NaN,NaN,29,Briarwood Rd,NaN,NaN,2,NaN,NaN,NaN,NaN,Unionville,relevant


In [11]:
data_profile = data_profiling(data_set)
# data_profile()
# data_profile.cat_col
# data_profile.float_column
# data_set[data_profile.int_column]

# target_variable = 'SalePrice'
target_variable = 'sold price\n(million)'

X = data_set[data_profile.int_column].drop(columns =[target_variable])
y = house_price[target_variable]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



KeyError: "['sold price\\n(million)'] not found in axis"

### 

# <a id='toc1_'></a>[model](#toc0_)

In [4]:
data_profile.int_column.remove(target_variable)

preprocessor = ColumnTransformer(transformers = [
                                   #         ('text', TfidfVectorizer(max_features=500), text_col),
                                               ('int_col',StandardScaler(),data_profile.int_column),
                                          #('int_col',StandardScaler(), data_profile.int_column[3:-1]),
                                          # ('int_col',StandardScaler(), data_profile.int_column[0:3]),
                                        #    ('cat_col', OneHotEncoder(), cat_col),
                                            ]) 

In [5]:
import contextlib
import io

f = io.StringIO()
with contextlib.redirect_stdout(f):
    print('h')

In [8]:

best_model ={}
results    = {}

for model_name, model_infor in model_parameters_regressor.items():
    pipeline = Pipeline([('preprocessor',preprocessor),
                         ('regressor',model_infor['model'])])
    grid_search = GridSearchCV(estimator=pipeline,
                                param_grid = model_infor['params'],
                                cv = 5, # 5 fold cv
                                scoring = 'neg_root_mean_squared_error', # neg_root_mean_squared_error,neg_mean_squared_error,r2
                                verbose =0, # not to display output
                                n_jobs = -1 # use all of cores
                                )
    

    model_trained = grid_search.fit(X_train, y_train)
    
    best_model[model_name] = model_trained.best_estimator_
    #print(f"Best Parameters for {model_name}: {grid_search.best_params_}")
    y_pred = model_trained.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)  # Test RMSE
    r2 = r2_score(y_test, y_pred)  # R^2 Score
    mae = mean_absolute_error(y_test, y_pred) # Mean Absolute Error
    mape = (abs((y_test - y_pred) / y_test).mean()) * 100 # Mean Absolute Percentage Error (MAPE)
   # msle = mean_squared_log_error(y_test, y_pred)


    results[model_name] = {'MAE': mae,
                           'MAPE': mape,
                        #    'MSLE':msle,
                           'RMSE': rmse,
                           'R2': r2,
                           'Best Params': grid_search.best_params_
                           }

    results_df = pd.DataFrame(results).T  # Transpose for readability

results_df

/Users/jianbinlin/Documents/venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1196: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
/Users/jianbinlin/Documents/venv/lib/python3.10/site-packages/sklearn/neural_network/_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
/Users/jianbinlin/Documents/venv/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/jianbinlin/Documents/venv/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/jianbinlin/Documents/venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1196: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
/Users/jianbinlin/Documents/venv/lib/python3.10/si

,MAE,MAPE,RMSE,R2,Best Params
decisiontree_regressor,25971.748288,15.950407,39733.121882,0.794178,"{'regressor__max_depth': None, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 2}"
randomforest_regressor,21106.875571,12.764414,37573.798338,0.815941,"{'regressor__bootstrap': True, 'regressor__criterion': 'absolute_error', 'regressor__max_depth': None, 'regressor__max_features': None, 'regressor__max_samples': 0.1, 'regressor__n_estimators': 30}"
adaboost_regressor,23926.685036,16.658353,34475.661734,0.845043,"{'regressor__learning_rate': 0.5, 'regressor__loss': 'linear', 'regressor__n_estimators': 100}"
catboost_regressor,16870.470592,9.803082,29037.696729,0.890071,"{'regressor__depth': 6, 'regressor__l2_leaf_reg': 1, 'regressor__learning_rate': 0.03}"
xgboost_regressor,20089.851723,11.698609,30388.531194,0.879606,"{'regressor__colsample_bytree': 1, 'regressor__eta': 0.5, 'regressor__gamma': 0, 'regressor__max_depth': 5, 'regressor__min_child_weight': 1, 'regressor__subsample': 1.0}"
lgb_regressor,17538.01745,10.196698,28218.145601,0.896189,"{'regressor__bagging_fraction': 0.5, 'regressor__colsample_bytree': None, 'regressor__feature_fraction': 0.5, 'regressor__learning_rate': 0.1, 'regressor__max_depth': 5, 'regressor__min_child_samples': 10, 'regressor__num_leaves': 31}"
mlp_regressor,29815.686476,17.426952,49964.698144,0.674529,"{'regressor__activation': 'tanh', 'regressor__alpha': 1e-05, 'regressor__early_stopping': True, 'regressor__hidden_layer_sizes': (50,), 'regressor__learning_rate_init': 0.01, 'regressor__max_iter': 200, 'regressor__n_iter_no_change': 10, 'regressor__solver': 'sgd'}"
